In [143]:
import pandas as pd
import numpy as np
from pybaseball import statcast_pitcher, playerid_lookup, statcast_batter

In [74]:
ids = pd.read_csv("mlbids.csv")

In [77]:
# get pitcher data on yamamoto in just the 2025 season
START = "2025-03-01"
END   = "2025-11-20"
player = playerid_lookup('yamamoto', 'yoshinobu')
print(player)

  name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0  yamamoto  yoshinobu     808967  yamay001  yamamyo01          33825   

   mlb_played_first  mlb_played_last  
0            2024.0           2025.0  


In [79]:
df = statcast_pitcher(START, END, 808967)

Gathering Player Data


In [80]:
df

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
0,FS,2025-11-01,92.1,-2.02,5.15,"Yamamoto, Yoshinobu",672386,808967,grounded_into_double_play,hit_into_play,...,NaN,2.33,0.48,0.48,36.9,8.375233,-7.664043,17.979522,48.911447,37.014581
1,CU,2025-11-01,80.3,-1.70,5.45,"Yamamoto, Yoshinobu",672386,808967,NaN,called_strike,...,NaN,4.85,-1.36,-1.36,48.1,NaN,NaN,NaN,NaN,NaN
2,FC,2025-11-01,92.8,-1.91,5.24,"Yamamoto, Yoshinobu",672386,808967,NaN,foul,...,NaN,2.04,-0.23,-0.23,39.1,14.162806,5.694338,22.383742,47.652657,23.055298
3,FS,2025-11-01,90.9,-1.97,5.26,"Yamamoto, Yoshinobu",680718,808967,walk,blocked_ball,...,NaN,2.74,0.69,-0.69,40.1,NaN,NaN,NaN,NaN,NaN
4,FS,2025-11-01,90.8,-2.02,5.27,"Yamamoto, Yoshinobu",680718,808967,NaN,ball,...,NaN,2.30,0.23,-0.23,38.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3435,FS,2025-03-04,90.7,-1.60,5.59,"Yamamoto, Yoshinobu",680574,808967,NaN,blocked_ball,...,NaN,2.55,0.95,0.95,NaN,NaN,NaN,NaN,NaN,NaN
3436,FF,2025-03-04,95.8,-1.58,5.66,"Yamamoto, Yoshinobu",680574,808967,NaN,foul,...,NaN,1.11,0.60,0.60,NaN,NaN,NaN,NaN,NaN,NaN
3437,CU,2025-03-04,75.4,-1.27,5.89,"Yamamoto, Yoshinobu",680574,808967,NaN,ball,...,NaN,5.49,-0.68,-0.68,NaN,NaN,NaN,NaN,NaN,NaN
3438,FF,2025-03-04,96.8,-1.62,5.75,"Yamamoto, Yoshinobu",670770,808967,double,hit_into_play,...,NaN,0.88,0.73,-0.73,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# get only regular and post season games
if "game_type" in df.columns:
    df = df[df["game_type"].isin(["R", "P"])].copy()

In [85]:
df = pd.merge(df, ids, left_on='batter', right_on='MLBID', how='left').copy()

In [87]:
# rename pitcher team score and opponent team score
df["pitcher_team"] = df["fld_score"]
df["opponent"] = df["bat_score"]

In [89]:
# change types
df["balls"]   = df["balls"].astype(int)
df["strikes"] = df["strikes"].astype(int)
df["outs"]    = df["outs_when_up"].astype(int)
df["inning"]  = df["inning"].astype(int)
df["top_bot"] = (df["inning_topbot"] == "Top").astype(int)

# change name of batter name and id column
df["batter_name"] = df["PLAYERNAME"]
df["batter_id"] = df["batter"]

In [91]:
# change pitcher and batter hand column names to be more intuitive
df["p_hand"] = df["p_throws"]
df["b_hand"] = df["stand"]

df["same_hand_matchup"] = (df["p_hand"] == df["b_hand"]).astype(int)

In [93]:
# total number of runners on bases
df["num_runners_onb"] = df[["on_1b", "on_2b", "on_3b"]].sum(axis=1)

df[['on_3b', 'on_2b', 'on_1b']] = df[['on_3b', 'on_2b', 'on_1b']].notna()

In [95]:
# get previous pitch data
df["prev_pitch_type"] = (
    df.groupby(["game_pk", "at_bat_number"])["pitch_type"].shift(1)
)
df["prev_pitch_result"] = (
    df.groupby(["game_pk", "at_bat_number"])["description"]
      .shift(1)
)

In [97]:
# get number of pitches thrown in that game
df["p_pitch_count"] = df.groupby(["game_pk"])["pitch_type"].cumcount() + 1

In [99]:
cols = [
    'game_pk', 'game_date', 'at_bat_number',
    'pitch_number', 'pitch_type',
    'batter_name', 'batter_id',
    'p_hand', 'b_hand',
    'same_hand_matchup',
    'balls', 'strikes',
    'outs', 'on_3b',
    'on_2b','on_1b',
    'inning', 'top_bot',
    'p_pitch_count',
    'pitcher_team', 'opponent',
    'prev_pitch_type', 'prev_pitch_result'
]

In [118]:
df = df[cols].copy()
df = df.dropna()

In [120]:
df.to_csv('yama_pitching_data', index=False)

In [122]:
df

,game_pk,game_date,at_bat_number,pitch_number,pitch_type,batter_name,batter_id,p_hand,b_hand,same_hand_matchup,...,on_3b,on_2b,on_1b,inning,top_bot,p_pitch_count,pitcher_team,opponent,prev_pitch_type,prev_pitch_result
1,776185,2025-09-25,57,5,SI,Blaze Alexander,677942,R,R,1,...,False,False,True,6,0,2,8,0,SL,hit_into_play
2,776185,2025-09-25,57,4,FF,Blaze Alexander,677942,R,R,1,...,False,False,True,6,0,3,8,0,SI,ball
3,776185,2025-09-25,57,3,FF,Blaze Alexander,677942,R,R,1,...,False,False,True,6,0,4,8,0,FF,foul
4,776185,2025-09-25,57,2,FS,Blaze Alexander,677942,R,R,1,...,False,False,True,6,0,5,8,0,FF,ball
5,776185,2025-09-25,57,1,SI,Blaze Alexander,677942,R,R,1,...,False,False,True,6,0,6,8,0,FS,ball
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,778563,2025-03-18,5,1,FF,Seiya Suzuki,673548,R,R,1,...,False,False,True,1,0,67,0,0,FS,ball
2787,778563,2025-03-18,4,5,FF,Ian Happ,664023,R,L,0,...,False,False,False,1,0,69,0,0,FF,ball
2788,778563,2025-03-18,4,4,FF,Ian Happ,664023,R,L,0,...,False,False,False,1,0,70,0,0,FF,called_strike
2789,778563,2025-03-18,4,3,FF,Ian Happ,664023,R,L,0,...,False,False,False,1,0,71,0,0,FF,called_strike


In [153]:
all_rows = []

for bid in unique_batters: #Get all unique batters Yamamoto faced
    bf = statcast_batter('2025-03-01', '2025-10-01', bid)
    if bf.empty:
        continue

    bf = bf[bf['events'].notna()].copy() #Remove entries that had no results

    #Mark hits, etc.
    tb_map = {'single':1,'double':2,'triple':3,'home_run':4}

    bf['is_hit'] = bf['events'].isin(tb_map.keys())
    bf['total_bases'] = bf['events'].map(tb_map).fillna(0)
    bf['is_walk'] = bf['events'].isin(['walk','intent_walk'])
    bf['is_hbp']  = bf['events'] == 'hit_by_pitch'
    bf['is_sf']   = bf['events'] == 'sac_fly'

    bf['AB'] = (~bf['is_walk'] & ~bf['is_hbp'] & ~bf['is_sf']).astype(int)

    grouped = bf.groupby('pitch_type').agg( #Calculate stats vs Yamamoto
        AB=('AB','sum'),
        H=('is_hit','sum'),
        BB=('is_walk','sum'),
        HBP=('is_hbp','sum'),
        SF=('is_sf','sum'),
        TB=('total_bases','sum')
    ).reset_index()

    grouped['batter_id'] = bid
    all_rows.append(grouped)

#Combine results
full = pd.concat(all_rows, ignore_index=True)

full['AVG'] = full['H'] / full['AB'].replace(0, pd.NA)
full['OBP'] = (full['H'] + full['BB'] + full['HBP']) / \
              (full['AB'] + full['BB'] + full['HBP'] + full['SF']).replace(0, pd.NA)
full['SLG'] = full['TB'] / full['AB'].replace(0, pd.NA)

final_df = full.pivot(
    index='batter_id',
    columns='pitch_type',
    values=['AVG','OBP','SLG']
)

final_df.columns = [f"{stat}_{ptype}" for (stat, ptype) in final_df.columns]
final_df = final_df.reset_index()

#Add the names to table, given batter ID
name_map = (
    df[['batter_id','batter_name']]
      .drop_duplicates()
      .set_index('batter_id')['batter_name']
      .to_dict()
)

final_df['batter_name'] = final_df['batter_id'].map(name_map)

cols = ['batter_name','batter_id'] + [c for c in final_df.columns if c not in ['batter_name','batter_id']]
final_df = final_df[cols]




Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

In [154]:
final_df

,batter_name,batter_id,AVG_CH,AVG_CS,AVG_CU,AVG_EP,AVG_FA,AVG_FC,AVG_FF,AVG_FO,...,SLG_FO,SLG_FS,SLG_KC,SLG_KN,SLG_PO,SLG_SC,SLG_SI,SLG_SL,SLG_ST,SLG_SV
0,Martin Maldonado,455117,0.0,1.0,0.090909,NaN,<NA>,0.272727,0.2,NaN,...,NaN,0.5,0.0,NaN,NaN,NaN,0.285714,0.275862,0.5,NaN
1,Andrew McCutchen,457705,0.289474,NaN,0.236842,1.0,NaN,0.384615,0.184713,NaN,...,NaN,0.692308,0.5,NaN,NaN,NaN,0.311927,0.367089,0.175,<NA>
2,Tommy Pham,502054,0.310345,0.0,0.142857,NaN,0.0,0.272727,0.239316,NaN,...,NaN,0.727273,0.0,NaN,NaN,NaN,0.354167,0.42623,0.102564,0.5
3,Paul Goldschmidt,502671,0.293103,NaN,0.285714,NaN,1.0,0.28,0.308725,NaN,...,NaN,0.3125,0.0,NaN,NaN,NaN,0.444444,0.426667,0.225806,0.333333
4,Travis D'Arnaud,518595,0.1875,NaN,0.0,NaN,NaN,0.058824,0.191781,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,0.361111,0.485714,0.647059,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Chase Meidroth,805367,0.152174,NaN,0.315789,NaN,0.0,0.263158,0.304636,NaN,...,NaN,0.333333,0.4,NaN,NaN,NaN,0.320388,0.231707,0.166667,0.0
205,Jacob Wilson,805779,0.410256,0.0,0.166667,NaN,0.0,0.314286,0.315315,NaN,...,NaN,0.461538,1.0,NaN,NaN,NaN,0.371069,0.641304,0.176471,0.0
206,Bryce Eldridge,805811,0.0,NaN,0.333333,NaN,NaN,0.0,0.083333,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,2.0,0.0,0.0,NaN
207,Matt Shaw,807713,0.261905,NaN,0.190476,0.0,0.0,0.304348,0.2,NaN,...,NaN,1.714286,0.0,NaN,NaN,NaN,0.253333,0.4,0.363636,0.4


In [155]:
final_df[:24]

,batter_name,batter_id,AVG_CH,AVG_CS,AVG_CU,AVG_EP,AVG_FA,AVG_FC,AVG_FF,AVG_FO,...,SLG_FO,SLG_FS,SLG_KC,SLG_KN,SLG_PO,SLG_SC,SLG_SI,SLG_SL,SLG_ST,SLG_SV
0,Martin Maldonado,455117,0.0,1.0,0.090909,NaN,<NA>,0.272727,0.2,NaN,...,NaN,0.5,0.0,NaN,NaN,NaN,0.285714,0.275862,0.5,NaN
1,Andrew McCutchen,457705,0.289474,NaN,0.236842,1.0,NaN,0.384615,0.184713,NaN,...,NaN,0.692308,0.5,NaN,NaN,NaN,0.311927,0.367089,0.175,<NA>
2,Tommy Pham,502054,0.310345,0.0,0.142857,NaN,0.0,0.272727,0.239316,NaN,...,NaN,0.727273,0.0,NaN,NaN,NaN,0.354167,0.42623,0.102564,0.5
3,Paul Goldschmidt,502671,0.293103,NaN,0.285714,NaN,1.0,0.28,0.308725,NaN,...,NaN,0.3125,0.0,NaN,NaN,NaN,0.444444,0.426667,0.225806,0.333333
4,Travis D'Arnaud,518595,0.1875,NaN,0.0,NaN,NaN,0.058824,0.191781,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,0.361111,0.485714,0.647059,0.0
5,DJ LeMahieu,518934,0.153846,NaN,0.142857,0.0,NaN,0.272727,0.361111,NaN,...,NaN,0.5,0.0,NaN,NaN,NaN,0.095238,0.466667,0.428571,0.0
6,Wilmer Flores,527038,0.195122,NaN,0.190476,<NA>,NaN,0.125,0.266234,NaN,...,NaN,0.0,0.375,NaN,NaN,NaN,0.451613,0.333333,0.387097,0.0
7,Marcell Ozuna,542303,0.137255,NaN,0.137931,NaN,NaN,0.26087,0.247934,0.0,...,0.0,0.238095,0.142857,NaN,NaN,NaN,0.481481,0.37037,0.409091,0.0
8,Jon Berti,542932,0.0,NaN,0.2,0.5,1.0,0.2,0.25,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,0.142857,0.125,0.555556,NaN
9,Marcus Semien,543760,0.159091,NaN,0.235294,NaN,NaN,0.255814,0.228916,NaN,...,NaN,0.0,0.777778,NaN,NaN,NaN,0.408602,0.329412,0.416667,1.0
